In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.ndimage import zoom, gaussian_gradient_magnitude, rotate
from scipy.optimize import curve_fit
import os

In [ ]:
# Constants
GYROMAGNETIC_RATIO = 42.577e6  # Gyromagnetic constant in Hz/T
DEAD_TIME = 3e-3  # Dead time in seconds
FIELD_STRENGTHS = np.arange(0.5, 9.5, 0.5)  # Magnetic field range (0.5 to 9 T)
TE_INCREMENT = 0.4e-3  # TE increment in seconds
FLIP_ANGLE_INCREMENT = 0.2  # Flip angle increment in degrees
TISSUES = ["WM", "GM", "CSF"]

# Tissue-specific parameters for T1 and T2* calculations
T1_PARAMS = {
    "WM": {"alpha_t1": 0.00071, "beta_t1": 0.382},
    "GM": {"alpha_t1": 0.00116, "beta_t1": 0.376},
    "CSF": {"alpha_t1": 4.329, "beta_t1": 0}
}

T2_STAR_PARAMS = {
    "WM": {"alpha_t2s": 0.090, "beta_t2s": 0.142},
    "GM": {"alpha_t2s": 0.064, "beta_t2s": 0.132},
    "CSF": {"alpha_t2s": 0.1, "beta_t2s": 0}
}

# Additional delta parameters based on B0
DELTA_T1 = {
    "WM": lambda b0: (2 * b0 + 18) * 0.001,  # in seconds
    "GM": lambda b0: (5 * b0 + 54) * 0.001,  # in seconds
    "CSF": lambda b0: 0.2  # 200 ms in seconds (δ = 0.2 s)
}

DELTA_T2_STAR = {
    "WM": lambda b0: (1.5 * b0) * 0.001,  # in seconds
    "GM": lambda b0: (1.5 * b0) * 0.001,  # in seconds
    "CSF": lambda b0: 0.003  # 3 ms in seconds
}

In [ ]:
# Function to compute T1 relaxation time (in seconds)
def compute_t1(b0, tissue):
    params = T1_PARAMS[tissue]
    alpha_t1, beta_t1 = params["alpha_t1"], params["beta_t1"]
    delta_t1 = DELTA_T1[tissue](b0)
    t1 = alpha_t1 * (GYROMAGNETIC_RATIO * b0) ** beta_t1 + delta_t1
    return t1  # in seconds


In [ ]:
# Function to compute T2* relaxation time (in seconds)
def compute_t2_star(b0, tissue):
    params = T2_STAR_PARAMS[tissue]
    alpha_t2s, beta_t2s = params["alpha_t2s"], params["beta_t2s"]
    delta_t2s = DELTA_T2_STAR[tissue](b0)
    t2_star = alpha_t2s * np.exp(-beta_t2s * b0) + delta_t2s
    return t2_star  # in seconds

In [ ]:
# Function to compute Bandwidth (BW)
def compute_bw(te):
    # Ensure TE is in seconds
    return 1 / (2 * te - DEAD_TIME)

In [ ]:
# Function to compute SNR for a tissue
def compute_snr(b0, tissue, tr, te, flip_angle):
    t1 = compute_t1(b0, tissue)  # in seconds
    t2_star = compute_t2_star(b0, tissue)  # in seconds
    bw = compute_bw(te)  # in Hz
    flip_angle_rad = np.radians(flip_angle)
    numerator = b0 * 1e3 * np.sin(flip_angle_rad) * np.exp(-te / t2_star)
    denominator = np.sqrt(bw) * (1 - np.cos(flip_angle_rad) * np.exp(-tr / t1))
    snr = numerator * (1 - np.exp(-tr / t1)) / denominator
    return snr


In [ ]:
def compute_te_tr_t1(b0):
    t2_star_wm = compute_t2_star(b0, "WM")  # in seconds
    te = t2_star_wm / 8
    tr = 2 * te
    return te, tr

In [ ]:
# Function to generate SNR maps
def generate_snr_maps(field_strengths, tissue_types, weighted_type):
    snr_maps = {}
    for b0 in field_strengths:
        snr_maps[b0] = {}
        if weighted_type == "T1":
            te, tr = compute_te_tr_t1(b0)
            optimal_flip_angle, _, _ = find_optimal_flip_angle(b0, "WM", "GM", tr, te)
            for tissue in tissue_types:
                snr_maps[b0][tissue] = compute_snr(b0, tissue, tr, te, optimal_flip_angle)
        elif weighted_type == "T2*":
            te, tr = compute_te_tr_t2_star(b0)
            for tissue in tissue_types:
                snr_maps[b0][tissue] = compute_snr(b0, tissue, tr, te, 90)  # Fixed flip angle of 90 degrees
        snr_maps[b0]['TR'] = tr  # Store TR for potential use
        snr_maps[b0]['TE'] = te  # Store TE for potential use
    return snr_maps



In [ ]:
# Function to find optimal TE for T2*-weighted scans
def find_optimal_te(b0, tissue1, tissue2):
    te_values = np.arange(0.4, 200, TE_INCREMENT)
    contrasts = []
    for te in te_values:
        snr1 = compute_snr(b0, tissue1, tr=1000, te=te, flip_angle=90)
        snr2 = compute_snr(b0, tissue2, tr=1000, te=te, flip_angle=90)
        contrast = abs(snr1 - snr2)
        contrasts.append(contrast)

    optimal_te = te_values[np.argmax(contrasts)]
    return optimal_te, te_values, contrasts

In [ ]:
# Function to find optimal flip angle for T1-weighted scans
def find_optimal_flip_angle(b0, tissue1, tissue2, tr, te):
    flip_angles = np.arange(0.2, 180, FLIP_ANGLE_INCREMENT)
    contrasts = []
    for angle in flip_angles:
        snr1 = compute_snr(b0, tissue1, tr, te, flip_angle=angle)
        snr2 = compute_snr(b0, tissue2, tr, te, flip_angle=angle)
        contrast = abs(snr1 - snr2)
        contrasts.append(contrast)

    optimal_angle = flip_angles[np.argmax(contrasts)]
    return optimal_angle, flip_angles, contrasts

In [ ]:
# Function to add Gaussian noise to the signal
def add_noise(signal, b0):
    noise_variance = b0 * np.mean(signal)
    noise = np.random.normal(0, np.sqrt(noise_variance), signal.shape)
    return signal + noise

In [ ]:
# Function to resample data to a new resolution
def resample_data(data, new_voxel_size):
    original_voxel_size = 1.0  # Assuming original voxel size is 1mm
    scale_factors = [new_voxel_size / original_voxel_size] * 3
    resampled_data = zoom(data, scale_factors, order=1)
    return resampled_data

In [ ]:
# Function to compute gradient entropy (gEn)
def compute_gradient_entropy(image, threshold=0):
    gradient_magnitude = gaussian_gradient_magnitude(image, sigma=1)
    gradient_magnitude = gradient_magnitude[image > threshold]  # Remove background
    hist, _ = np.histogram(gradient_magnitude, bins=256, density=True)
    hist = hist[hist > 0]
    return -np.sum(hist * np.log2(hist))

In [ ]:
# Function to compute sharpness
def compute_sharpness_profile(image, angle=90):
    rotated_image = rotate(image, angle, reshape=False)
    line_profile = rotated_image[rotated_image.shape[0] // 2, :]
    gradients = np.gradient(line_profile)

    # Define sigmoid function for fitting
    def sigmoid(x, L, x0, k, b):
        return L / (1 + np.exp(-k * (x - x0))) + b

    # Fit sigmoid to line profile
    x_data = np.arange(len(line_profile))
    try:
        popt, _ = curve_fit(sigmoid, x_data, line_profile, p0=[1, len(line_profile) / 2, 1, 0])
        sharpness = popt[2]  # "k" parameter indicates steepness
    except RuntimeError:
        sharpness = np.max(gradients) - np.min(gradients)

    return sharpness, line_profile

In [ ]:
# Main function
def main():
    # Load NIfTI data
    patient_files = ["/content/Patient1.nii.gz", "/content/Patient2.nii.gz"]
    for patient_id, file_path in enumerate(patient_files, start=1):
        img = nib.load(file_path)
        data = img.get_fdata()
        print(f"Loaded data for Patient {patient_id}")

        # Extract tissue maps
        gm_map = np.isin(data, [1, 2]).astype(np.float32)
        wm_map = (data == 3).astype(np.float32)
        csf_map = (data == 4).astype(np.float32)

        # Generate SNR maps
        snr_maps = generate_snr_maps(FIELD_STRENGTHS, TISSUES, 'T1')

        # Add noise to SNR maps
        b0 = 3.0  # Example field strength
        noisy_snr_maps = {tissue: add_noise(snr_maps[b0][tissue], b0) for tissue in TISSUES}

        # Compute Gradient Entropy and Sharpness
        for tissue, tissue_map in zip(TISSUES, [wm_map, gm_map, csf_map]):
            gEn = compute_gradient_entropy(tissue_map[:, :, 90])
            print(f"Gradient Entropy for {tissue}: {gEn:.4f}")
            sharpness, profile = compute_sharpness_profile(tissue_map[:, :, 90])
            print(f"Sharpness for {tissue}: {sharpness:.4f}")

if __name__ == "__main__":
    main()


In [ ]:
# Function to find optimal TE for T2*-weighted scans
def find_optimal_te(b0, tissue1, tissue2):
    te_values = np.arange(0.4e-3, 200e-3, TE_INCREMENT)  # TE in seconds
    contrasts = []
    for te in te_values:
        s_tissue1 = np.exp(-te / compute_t2_star(b0, tissue1))
        s_tissue2 = np.exp(-te / compute_t2_star(b0, tissue2))
        contrast = abs(s_tissue1 - s_tissue2)
        contrasts.append(contrast)
    optimal_te = te_values[np.argmax(contrasts)]
    return optimal_te, te_values, contrasts


# Function to compute TE and TR for T2*-weighted scans
def compute_te_tr_t2_star(b0):
    optimal_te, _, _ = find_optimal_te(b0, "WM", "GM")
    tr = 2 * optimal_te
    return optimal_te, tr

# Function to find optimal flip angle for T1-weighted scans
def find_optimal_flip_angle(b0, tissue1, tissue2, tr, te):
    flip_angles = np.arange(0.2, 180, FLIP_ANGLE_INCREMENT)
    contrasts = []
    for angle in flip_angles:
        flip_angle_rad = np.radians(angle)
        t1_tissue1 = compute_t1(b0, tissue1)
        t1_tissue2 = compute_t1(b0, tissue2)
        e1_tissue1 = np.exp(-tr / t1_tissue1)
        e1_tissue2 = np.exp(-tr / t1_tissue2)
        s_tissue1 = (1 - e1_tissue1) * np.sin(flip_angle_rad) / (1 - np.cos(flip_angle_rad) * e1_tissue1)
        s_tissue2 = (1 - e1_tissue2) * np.sin(flip_angle_rad) / (1 - np.cos(flip_angle_rad) * e1_tissue2)
        contrast = abs(s_tissue1 - s_tissue2)
        contrasts.append(contrast)
    optimal_angle = flip_angles[np.argmax(contrasts)]
    return optimal_angle, flip_angles, contrasts

# Function to add Gaussian noise to the signal
def add_noise(signal, sigma_n):
    noise = np.random.normal(0, sigma_n, signal.shape)
    return signal + noise

# Function to resample data to a new resolution
def resample_data(data, new_voxel_size, new_slice_thickness):
    original_voxel_size = 1.0  # in mm
    # Compute scale factors for in-plane and slice directions
    scale_factors = [original_voxel_size / new_voxel_size,
                     original_voxel_size / new_voxel_size,
                     original_voxel_size / new_slice_thickness]
    resampled_data = zoom(data, scale_factors, order=1)
    return resampled_data

# Function to compute sharpness profile
def compute_sharpness_profile(image, angle=90):
    rotated_image = rotate(image, angle, reshape=False)
    # Extract the central line profile
    line_profile = rotated_image[rotated_image.shape[0] // 2, :]
    # Compute gradients
    gradients = np.gradient(line_profile)
    # Define sigmoid function for fitting
    def sigmoid(x, L, x0, k, b):
        return L / (1 + np.exp(-k * (x - x0))) + b
    # Fit sigmoid to line profile
    x_data = np.arange(len(line_profile))
    p0 = [np.max(line_profile), len(line_profile) / 2, 1, np.min(line_profile)]
    try:
        popt, _ = curve_fit(sigmoid, x_data, line_profile, p0=p0)
        sharpness = popt[2]  # 'k' parameter indicates steepness
    except RuntimeError:
        sharpness = np.max(gradients) - np.min(gradients)
    return sharpness, line_profile

# Function to plot T1 vs B0
def plot_t1_vs_b0(field_strengths):
    plt.figure(figsize=(8, 6))
    for tissue in TISSUES:
        t1_values = [compute_t1(b0, tissue) * 1e3 for b0 in field_strengths]  # Convert to ms
        plt.plot(field_strengths, t1_values, marker='o', label=tissue)
    plt.xlabel('Magnetic Field Strength (T)')
    plt.ylabel('T1 Relaxation Time (ms)')
    plt.title('T1 Relaxation Time vs. Magnetic Field Strength')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Function to plot T2* vs B0
def plot_t2_star_vs_b0(field_strengths):
    plt.figure(figsize=(8, 6))
    for tissue in TISSUES:
        t2_star_values = [compute_t2_star(b0, tissue) * 1e3 for b0 in field_strengths]  # Convert to ms
        plt.plot(field_strengths, t2_star_values, marker='s', label=tissue)
    plt.xlabel('Magnetic Field Strength (T)')
    plt.ylabel('T2* Relaxation Time (ms)')
    plt.title('T2* Relaxation Time vs. Magnetic Field Strength')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Function to plot Contrast vs TE for T2*-weighted scans
def plot_contrast_vs_te(b0):
    optimal_te, te_values, contrasts = find_optimal_te(b0, "WM", "GM")
    plt.figure(figsize=(8, 6))
    plt.plot(te_values * 1e3, contrasts, label='Contrast (WM vs GM)')
    plt.axvline(optimal_te * 1e3, color='r', linestyle='--', label=f'Optimal TE: {optimal_te*1e3:.2f} ms')
    plt.xlabel('Echo Time (ms)')
    plt.ylabel('Contrast')
    plt.title(f'Contrast vs. TE for B0 = {b0} T')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    return optimal_te

# Function to plot Contrast vs Flip Angle for T1-weighted scans
def plot_contrast_vs_flip_angle(b0):
    te, tr = compute_te_tr_t1(b0)
    optimal_flip_angle, flip_angles, contrasts = find_optimal_flip_angle(b0, "WM", "GM", tr, te)
    plt.figure(figsize=(8, 6))
    plt.plot(flip_angles, contrasts, label='Contrast (WM vs GM)')
    plt.axvline(optimal_flip_angle, color='r', linestyle='--', label=f'Optimal Flip Angle: {optimal_flip_angle:.2f}°')
    plt.xlabel('Flip Angle (degrees)')
    plt.ylabel('Contrast')
    plt.title(f'Contrast vs. Flip Angle for B0 = {b0} T')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    return optimal_flip_angle

# Function to plot SNR vs B0 for each tissue
def plot_snr_vs_b0(snr_maps, tissue):
    b0_values = sorted(snr_maps.keys())
    snr_values = [snr_maps[b0][tissue] for b0 in b0_values]
    plt.plot(b0_values, snr_values, marker='^', label=tissue)
    plt.xlabel('Magnetic Field Strength (T)')
    plt.ylabel('SNR (au)')
    plt.title(f'SNR vs. B0 for {tissue}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

# Function to plot SNR across all tissues
def plot_all_snr_vs_b0(snr_maps):
    plt.figure(figsize=(8, 6))
    for tissue in TISSUES:
        plot_snr_vs_b0(snr_maps, tissue)
    plt.title('SNR vs. Magnetic Field Strength for All Tissues')
    plt.legend()
    plt.show()

# Function to plot MRI slice
def plot_mri_slice(slice_data, title='MRI Slice', cmap='gray'):
    plt.figure(figsize=(6, 6))
    plt.imshow(slice_data, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Function to plot Gradient Entropy
def plot_gradient_entropy(gEn_values, tissues):
    plt.figure(figsize=(8, 6))
    for tissue in tissues:
        plt.plot(FIELD_STRENGTHS, gEn_values[tissue], marker='o', label=tissue)
    plt.xlabel('Magnetic Field Strength (T)')
    plt.ylabel('Gradient Entropy (gEn)')
    plt.title('Gradient Entropy vs. Magnetic Field Strength')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Function to plot Sharpness vs B0
def plot_sharpness(sharpness_values, tissue):
    plt.figure(figsize=(8, 6))
    plt.plot(FIELD_STRENGTHS, sharpness_values[tissue], marker='s', label=tissue)
    plt.xlabel('Magnetic Field Strength (T)')
    plt.ylabel('Sharpness')
    plt.title(f'Sharpness vs. Magnetic Field Strength for {tissue}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Function to calculate the resampled slice index
def get_resampled_slice_index(original_slice_index, original_voxel_size_z, new_slice_thickness, num_slices_resampled):
    z_position = original_slice_index * original_voxel_size_z  # in mm
    slice_index_resampled = int(round(z_position / new_slice_thickness))
    # Ensure the index is within bounds
    slice_index_resampled = min(max(slice_index_resampled, 0), num_slices_resampled - 1)
    return slice_index_resampled

# Function to simulate MRI images with noise
def simulate_mri_images(snr_maps_b0, patient_maps, b0, slice_index=None):
    # Compute noise standard deviation
    sigma_n = 40 * (1 + np.log(b0))
    # For each tissue, compute Signal Intensity
    si_maps = {}
    for tissue in TISSUES:
        snr = snr_maps_b0[tissue]
        si = snr * sigma_n  # SI_tissue,B0 = SNR_tissue,B0 * σ_n
        si_maps[tissue] = si
    # Combine tissues into a single MRI image
    mri_image = np.zeros_like(patient_maps['WM'])
    for tissue in TISSUES:
        mri_image += si_maps[tissue] * patient_maps[tissue]
    # Add Gaussian noise with standard deviation σ_n
    mri_noisy = add_noise(mri_image, sigma_n)
    # Extract the desired slice, or the middle slice if slice_index is None
    if slice_index is None:
        slice_index = mri_noisy.shape[2] // 2
    slice_noisy = mri_noisy[:, :, slice_index]
    return slice_noisy

# Function to process and simulate MRI images for all patients
def process_patients(patient_files, field_strengths):
    # Initialize storage for evaluation metrics
    evaluation_metrics = {}
    for patient_id, file_path in enumerate(patient_files, start=1):
        img = nib.load(file_path)
        data = img.get_fdata()
        print(f"Loaded data for Patient {patient_id}: {file_path}")

        # Extract tissue maps
        gm_map = np.isin(data, [1, 2]).astype(np.float32)
        wm_map = (data == 3).astype(np.float32)
        csf_map = (data == 4).astype(np.float32)
        tissue_maps = {"WM": wm_map, "GM": gm_map, "CSF": csf_map}

        # Generate SNR maps for T1 and T2* weighted scans
        snr_maps_t1 = generate_snr_maps(field_strengths, TISSUES, weighted_type="T1")
        snr_maps_t2_star = generate_snr_maps(field_strengths, TISSUES, weighted_type="T2*")

        # Plot T1 and T2* vs B0
        print(f"Plotting T1 and T2* relaxation times for Patient {patient_id}")
        plot_t1_vs_b0(field_strengths)
        plot_t2_star_vs_b0(field_strengths)

        # Plot Contrast vs TE for T2*-weighted scans for each B0
        for b0 in field_strengths:
            print(f"Plotting Contrast vs TE for B0 = {b0} T")
            plot_contrast_vs_te(b0)

        # Plot Contrast vs Flip Angle for T1-weighted scans for each B0
        for b0 in field_strengths:
            print(f"Plotting Contrast vs Flip Angle for B0 = {b0} T")
            plot_contrast_vs_flip_angle(b0)

        # Plot SNR vs B0 for each tissue
        print(f"Plotting SNR vs B0 for T1-weighted scans for Patient {patient_id}")
        plot_all_snr_vs_b0(snr_maps_t1)
        print(f"Plotting SNR vs B0 for T2*-weighted scans for Patient {patient_id}")
        plot_all_snr_vs_b0(snr_maps_t2_star)

        # Simulate MRI images at different B0 and plot slice 90
        for b0 in field_strengths:
            print(f"Simulating MRI image for B0 = {b0} T, Patient {patient_id}")
            snr_maps_b0 = snr_maps_t2_star[b0]
            mri_slice = simulate_mri_images(snr_maps_b0, tissue_maps, b0, slice_index=90)
            plot_mri_slice(mri_slice, title=f'Patient {patient_id} - B0={b0}T - Slice 90')

        # Compute Gradient Entropy and Sharpness for slice 90
        gEn_values = {tissue: [] for tissue in TISSUES}
        sharpness_values = {tissue: [] for tissue in TISSUES}
        for b0 in field_strengths:
            print(f"Computing Gradient Entropy and Sharpness for B0 = {b0} T, Patient {patient_id}")
            # Simulate MRI image
            snr_maps_b0 = snr_maps_t2_star[b0]
            mri_slice = simulate_mri_images(snr_maps_b0, tissue_maps, b0, slice_index=90)
            # Compute Gradient Entropy
            for tissue in TISSUES:
                tissue_slice = tissue_maps[tissue][:, :, 90] * mri_slice
                gEn = compute_gradient_entropy(tissue_slice, threshold=0)
                gEn_values[tissue].append(gEn)
            # Compute Sharpness
            # Assuming we select WM tissue to compute sharpness
            tissue = "WM"
            tissue_slice = tissue_maps[tissue][:, :, 90] * mri_slice
            sharpness, _ = compute_sharpness_profile(tissue_slice, angle=90)
            sharpness_values[tissue].append(sharpness)

        # Store evaluation metrics
        evaluation_metrics[patient_id] = {
            "gEn": gEn_values,
            "sharpness": sharpness_values
        }

        # Plot Gradient Entropy
        print(f"Plotting Gradient Entropy for Patient {patient_id}")
        plot_gradient_entropy(gEn_values, TISSUES)

        # Plot Sharpness
        print(f"Plotting Sharpness for Patient {patient_id}")
        plot_sharpness(sharpness_values, "WM")

        # Resample data to different resolutions and slice thicknesses
        resolutions = [1.0, 2.0]  # in mm (in-plane)
        slice_thicknesses = [1.0, 2.0, 5.0, 10.0]  # in mm
        for new_voxel_size in resolutions:
            for new_slice_thickness in slice_thicknesses:
                print(f"Resampling data to {new_voxel_size}mm x {new_voxel_size}mm x {new_slice_thickness}mm")
                resampled_maps = {}
                for tissue in TISSUES:
                    resampled_maps[tissue] = resample_data(tissue_maps[tissue], new_voxel_size, new_slice_thickness)
                # Simulate MRI image with resampled data at a fixed B0, e.g., 3T
                b0 = 3.0
                if b0 in field_strengths:
                    original_voxel_size_z = 1.0 # Assuming original voxel size in z-direction is 1mm
                    num_slices_resampled = resampled_maps['WM'].shape[2]
                    resampled_slice_index = get_resampled_slice_index(90, original_voxel_size_z, new_slice_thickness, num_slices_resampled)
                    mri_slice_resampled = simulate_mri_images(snr_maps_b0, resampled_maps, b0, slice_index=resampled_slice_index)
                    plot_mri_slice(mri_slice_resampled, title=f'Patient {patient_id} - B0={b0}T - Resampled {new_voxel_size}x{new_voxel_size}x{new_slice_thickness}mm - Slice 90')

    return evaluation_metrics

# Main function
def main():
    # Define patient NIfTI file paths
    # Update these paths to the actual locations of your NIfTI files
    patient_files = ["/content/Patient1.nii.gz", "/content/Patient2.nii.gz"]

    # Check if files exist
    for file in patient_files:
        if not os.path.exists(file):
            print(f"File not found: {file}. Please check the path.")
            return

    # Process patients and collect evaluation metrics
    evaluation_metrics = process_patients(patient_files, FIELD_STRENGTHS)

    # Additional analysis or saving of evaluation metrics can be done here
    # For example, saving to a JSON or CSV file
    print("MRI Simulation and Analysis Completed.")

if __name__ == "__main__":
    main()


In [ ]:
# Function to generate SNR maps
def generate_snr_maps(field_strengths, tissue_types, weighted_type):
    snr_maps = {}
    for b0 in field_strengths:
        snr_maps[b0] = {}
        if weighted_type == "T1":
            te, tr = compute_te_tr_t1(b0)
            optimal_flip_angle, _, _ = find_optimal_flip_angle(b0, "WM", "GM", tr, te)
            for tissue in tissue_types:
                snr_maps[b0][tissue] = compute_snr(b0, tissue, tr, te, optimal_flip_angle)
        elif weighted_type == "T2*":
            te, tr = compute_te_tr_t2_star(b0)
            # Compute Ernst angle for WM
            t1_wm = compute_t1(b0, "WM")
            ernst_angle = np.degrees(np.arccos(np.exp(-tr / t1_wm)))
            for tissue in tissue_types:
                snr_maps[b0][tissue] = compute_snr(b0, tissue, tr, te, ernst_angle)
        snr_maps[b0]['TR'] = tr  # Store TR for potential use
        snr_maps[b0]['TE'] = te  # Store TE for potential use
    return snr_maps


# Function to compute Ernst angle for T2*-weighted scans
def compute_ernst_angle_t2_star(b0, tissue, tr):
    t1_tissue = compute_t1(b0, tissue)
    ernst_angle_rad = np.arccos(np.exp(-tr / t1_tissue))
    ernst_angle_deg = np.degrees(ernst_angle_rad)
    return ernst_angle_deg

# Function to resample data to a new resolution
def resample_data(data, new_voxel_size, new_slice_thickness):
    original_voxel_size = 1.0  # in mm
    # Compute scale factors for in-plane and slice directions
    scale_factors = [
        original_voxel_size / new_voxel_size,
        original_voxel_size / new_voxel_size,
        original_voxel_size / new_slice_thickness
    ]
    resampled_data = zoom(data, scale_factors, order=1)
    return resampled_data


# Function to plot Ernst Angle vs T1_tissue for T2*-weighted scans
def plot_ernst_angle_vs_t1(field_strengths):
    t1_values = []
    ernst_angles = []
    for b0 in field_strengths:
        tr = compute_te_tr_t2_star(b0)[1]  # Get TR for T2*-weighted scans
        t1_tissue = compute_t1(b0, "WM")
        t1_values.append(t1_tissue * 1e3)  # Convert to ms
        ernst_angle = compute_ernst_angle_t2_star(b0, "WM", tr)
        ernst_angles.append(ernst_angle)
    plt.figure(figsize=(8, 6))
    plt.plot(t1_values, ernst_angles, marker='o', label='WM')
    plt.xlabel('T1 Relaxation Time (ms)')
    plt.ylabel('Ernst Angle (degrees)')
    plt.title('Ernst Angle vs. T1 Relaxation Time for T2*-Weighted Scans')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Function to simulate MRI images with noise
def simulate_mri_images(snr_maps_b0, patient_maps, b0, slice_index):
    # Compute noise standard deviation
    sigma_n = 40 * (1 + np.log(b0))
    # For each tissue, compute Signal Intensity
    si_maps = {}
    for tissue in TISSUES:
        snr = snr_maps_b0[tissue]
        si = snr * sigma_n  # SI_tissue,B0 = SNR_tissue,B0 * σ_n
        si_maps[tissue] = si
    # Combine tissues into a single MRI image
    mri_image = np.zeros_like(patient_maps['WM'])
    for tissue in TISSUES:
        mri_image += si_maps[tissue] * patient_maps[tissue]
    # Add Gaussian noise with standard deviation σ_n
    mri_noisy = add_noise(mri_image, sigma_n)
    # Extract the desired slice
    slice_noisy = mri_noisy[:, :, slice_index]
    return slice_noisy


# Function to process and simulate MRI images for all patients
def process_patients(patient_files, field_strengths):
    # Initialize storage for evaluation metrics
    evaluation_metrics = {}
    for patient_id, file_path in enumerate(patient_files, start=1):
        img = nib.load(file_path)
        data = img.get_fdata()
        print(f"Loaded data for Patient {patient_id}: {file_path}")

        # Extract tissue maps
        gm_map = np.isin(data, [1, 2]).astype(np.float32)
        wm_map = (data == 3).astype(np.float32)
        csf_map = (data == 4).astype(np.float32)
        tissue_maps = {"WM": wm_map, "GM": gm_map, "CSF": csf_map}

        # Generate SNR maps for T1 and T2* weighted scans
        snr_maps_t1 = generate_snr_maps(field_strengths, TISSUES, weighted_type="T1")
        snr_maps_t2_star = generate_snr_maps(field_strengths, TISSUES, weighted_type="T2*")

        # Plot T1 and T2* vs B0
        print(f"Plotting T1 and T2* relaxation times for Patient {patient_id}")
        plot_t1_vs_b0(field_strengths)
        plot_t2_star_vs_b0(field_strengths)

        # Plot Ernst Angle vs T1_tissue for T2*-weighted scans
        print(f"Plotting Ernst Angle vs T1_tissue for Patient {patient_id}")
        plot_ernst_angle_vs_t1(field_strengths)

        # Plot Contrast vs TE for T2*-weighted scans for each B0
        for b0 in field_strengths:
            print(f"Plotting Contrast vs TE for B0 = {b0} T")
            plot_contrast_vs_te(b0)

        # Plot Contrast vs Flip Angle for T1-weighted scans for each B0
        for b0 in field_strengths:
            print(f"Plotting Contrast vs Flip Angle for B0 = {b0} T")
            plot_contrast_vs_flip_angle(b0)

        # Plot SNR vs B0 for each tissue
        print(f"Plotting SNR vs B0 for T1-weighted scans for Patient {patient_id}")
        plot_all_snr_vs_b0(snr_maps_t1)
        print(f"Plotting SNR vs B0 for T2*-weighted scans for Patient {patient_id}")
        plot_all_snr_vs_b0(snr_maps_t2_star)

        # Simulate MRI images at different B0 and plot slice 90
        original_slice_index = 90  # Original slice index
        for b0 in field_strengths:
            print(f"Simulating MRI image for B0 = {b0} T, Patient {patient_id}")
            snr_maps_b0 = snr_maps_t2_star[b0]
            mri_slice = simulate_mri_images(snr_maps_b0, tissue_maps, b0, slice_index=original_slice_index)
            plot_mri_slice(mri_slice, title=f'Patient {patient_id} - B0={b0}T - Slice {original_slice_index}')

        # Compute Gradient Entropy and Sharpness for slice 90
        gEn_values = {tissue: [] for tissue in TISSUES}
        sharpness_values = {tissue: [] for tissue in TISSUES}
        for b0 in field_strengths:
            print(f"Computing Gradient Entropy and Sharpness for B0 = {b0} T, Patient {patient_id}")
            # Simulate MRI image
            snr_maps_b0 = snr_maps_t2_star[b0]
            mri_slice = simulate_mri_images(snr_maps_b0, tissue_maps, b0, slice_index=original_slice_index)
            # Compute Gradient Entropy
            for tissue in TISSUES:
                tissue_slice = tissue_maps[tissue][:, :, original_slice_index] * mri_slice
                gEn = compute_gradient_entropy(tissue_slice, threshold=0)
                gEn_values[tissue].append(gEn)
            # Compute Sharpness
            # Assuming we select WM tissue to compute sharpness
            tissue = "WM"
            tissue_slice = tissue_maps[tissue][:, :, original_slice_index] * mri_slice
            sharpness, _ = compute_sharpness_profile(tissue_slice, angle=90)
            sharpness_values[tissue].append(sharpness)

        # Store evaluation metrics
        evaluation_metrics[patient_id] = {
            "gEn": gEn_values,
            "sharpness": sharpness_values
        }

        # Plot Gradient Entropy
        print(f"Plotting Gradient Entropy for Patient {patient_id}")
        plot_gradient_entropy(gEn_values, TISSUES)

        # Plot Sharpness
        print(f"Plotting Sharpness for Patient {patient_id}")
        plot_sharpness(sharpness_values, "WM")

        # Resample data to different resolutions and slice thicknesses
        resolutions = [1.0, 2.0]  # in mm (in-plane)
        slice_thicknesses = [1.0, 2.0, 5.0, 10.0]  # in mm
        original_voxel_size = 1.0  # in mm
        for new_voxel_size in resolutions:
            for new_slice_thickness in slice_thicknesses:
                print(f"Resampling data to {new_voxel_size}mm x {new_voxel_size}mm x {new_slice_thickness}mm")
                resampled_maps = {}
                for tissue in TISSUES:
                    resampled_maps[tissue] = resample_data(tissue_maps[tissue], new_voxel_size, new_slice_thickness)
                # Simulate MRI image with resampled data at a fixed B0, e.g., 3T
                b0 = 3.0
                if b0 in field_strengths:
                    snr_maps_b0 = snr_maps_t2_star[b0]
                    # Compute the resampled slice index
                    num_slices_resampled = resampled_maps['WM'].shape[2]
                    slice_index_resampled = get_resampled_slice_index(
                        original_slice_index, original_voxel_size, new_slice_thickness, num_slices_resampled
                    )
                    mri_slice_resampled = simulate_mri_images(
                        snr_maps_b0, resampled_maps, b0, slice_index=slice_index_resampled
                    )
                    plot_mri_slice(
                        mri_slice_resampled,
                        title=f'Patient {patient_id} - B0={b0}T - Resampled {new_voxel_size}x{new_voxel_size}x{new_slice_thickness}mm - Slice {slice_index_resampled}'
                    )

    return evaluation_metrics

# Main function
def main():
    # Define patient NIfTI file paths
    # Update these paths to the actual locations of your NIfTI files
    patient_files = ["/content/Patient1.nii.gz", "/content/Patient2.nii.gz"]

    # Check if files exist
    for file in patient_files:
        if not os.path.exists(file):
            print(f"File not found: {file}. Please check the path.")
            return

    # Process patients and collect evaluation metrics
    evaluation_metrics = process_patients(patient_files, FIELD_STRENGTHS)

    # Additional analysis or saving of evaluation metrics can be done here
    # For example, saving to a JSON or CSV file
    print("MRI Simulation and Analysis Completed.")

if __name__ == "__main__":
    main()
